In [1]:
import pandas as pd
import numpy as np

import urllib.request, urllib.parse, json
import pickle

# Import crop data

In [2]:
crop_data = pd.read_csv("/content/drive/My Drive/Capstone_ML_Guild/Capstone/Data_Collection/Geo_GW_APY_Data_Step2.csv")

In [3]:
crop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214471 entries, 0 to 214470
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   State                     214471 non-null  object 
 1   District                  214471 non-null  object 
 2   Lat                       214471 non-null  float64
 3   Long                      214471 non-null  float64
 4   GW_depth_Minimum          214471 non-null  float64
 5   GW_depth_Maximum          214471 non-null  float64
 6   Wells_depth_0_to_2        214471 non-null  int64  
 7   Wells_depth_2_to_5        214471 non-null  int64  
 8   Wells_depth_5_to_10       214471 non-null  int64  
 9   Wells_depth_10_to_20      214471 non-null  float64
 10  Wells_depth_20_to_40      214471 non-null  float64
 11  Wells_depth_40_and_above  214471 non-null  float64
 12  Crop_Year                 214471 non-null  int64  
 13  Season                    214471 non-null  o

In [4]:
crop_data

,State,District,Lat,Long,GW_depth_Minimum,GW_depth_Maximum,Wells_depth_0_to_2,Wells_depth_2_to_5,Wells_depth_5_to_10,Wells_depth_10_to_20,Wells_depth_20_to_40,Wells_depth_40_and_above,Crop_Year,Season,Crop,Area,Production
0,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Arecanut,3100.0,5200.0
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Other Kharif pulses,668.0,448.0
2,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Kharif,Rice,10779.0,31863.0
3,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Banana,1531.0,12073.0
4,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,0.04,4.56,29,4,0,0.0,0.0,0.0,2000,Whole Year,Cashewnut,80.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214466,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Rice,195960.0,580442.0
214467,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Summer,Sesamum,74080.0,58514.0
214468,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Whole Year,Sugarcane,4390.0,789614.0
214469,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,0.70,16.90,5,8,10,13.0,0.0,0.0,2014,Winter,Rice,496134.0,1348329.0


In [5]:
df_crop = crop_data.copy()

In [6]:
df_crop.columns

Index(['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production'],
      dtype='object')

In [7]:
df_crop.columns = ['State', 'District', 'Lat', 'Long', 'GW_depth_Minimum',
       'GW_depth_Maximum', 'Wells_depth_0_to_2', 'Wells_depth_2_to_5',
       'Wells_depth_5_to_10', 'Wells_depth_10_to_20', 'Wells_depth_20_to_40',
       'Wells_depth_40_and_above', 'Year', 'Season', 'Crop', 'Area',
       'Production']

In [8]:
df_crop['Season'] = df_crop['Season'].apply(lambda s: s.strip())
seasons = df_crop['Season'].unique()
seasons

array(['Kharif', 'Whole Year', 'Autumn', 'Rabi', 'Summer', 'Winter'],
      dtype=object)

In [9]:
df_crop['Lat'] = df_crop['Lat'].apply(lambda lat: np.round(lat, 5))
df_crop['Long'] = df_crop['Long'].apply(lambda lon: np.round(lon, 5))

In [10]:
# Mapping of crop seasons to its months
season_months = {
    'Whole Year': list(range(1, 13)),
    'Autumn': list(range(9, 12)),
    'Kharif': list(range(6, 11)),
    'Rabi': [11, 12, 1, 2, 3],
    'Summer': [3, 4, 5, 6],
    'Winter': [12, 1]
}


# Get Weather data from CSV or server



In [11]:
lat_long_values = df_crop[['Lat', 'Long']].values

In [12]:
lat_long_values = np.unique(lat_long_values, axis=0)
lat_long_values[:10]

array([[ 8.08783, 77.53841],
       [ 8.381  , 77.6132 ],
       [ 8.56224, 76.92684],
       [ 8.80545, 77.67275],
       [ 8.87875, 76.76535],
       [ 9.26485, 76.78995],
       [ 9.34366, 78.86305],
       [ 9.46214, 76.28424],
       [ 9.63726, 76.60583],
       [ 9.8139 , 77.00703]])

In [13]:
# NASA API Dictionary
# ALLSKY_SFC_SW_DWN - Average amount of solar radiation incident on a horizontal surface at the surface of the earth under all-sky conditions (MJ/m^2/day)
# PRECTOT - Precipitation (mm)
# PS - Surface Pressure (kPa)
# RH2M - Relative Humidity at 2m (g/kg)
# T2MDEW - Dew/Frost Point at 2m (C)
# T2M_MAX - Max. Temperature at 2m (C)
# T2M_MIN - Min. Temperature at 2m (C)
# WS2M - Wind Speed at 2m (m/s)

weather_data = {}

try:
  in_file = open('/content/drive/MyDrive/Capstone_ML_Guild/Capstone/Time_Series_Analysis_for_Weather/timeseries_weather_data.pkl', 'rb')
  weather_data = pickle.load(in_file)                     
  in_file.close()
  print(f"Weather data found - {len(weather_data)}")
except:
  weather_data = {}

  print('Fething data...')
  prev_percent_complete = 0

  for index, (lat, lon) in enumerate(lat_long_values):
    req_url = f"https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&identifier=SinglePoint&parameters=PRECTOT,RH2M,PS,T2M_MAX,T2M_MIN,T2MDEW,WS2M,ALLSKY_SFC_SW_DWN&startDate=1981&endDate=2019&userCommunity=AG&tempAverage=INTERANNUAL&outputList=ASCII&lat={lat}&lon={lon}"
    with urllib.request.urlopen(req_url) as url:
      data = json.loads(url.read().decode())
      if lat not in weather_data.keys():
        weather_data[lat] = {}
      if lon not in weather_data[lat].keys():
        weather_data[lat][lon] = {}
      weather_data[lat][lon] = data['features'][0]['properties']['parameter']

    percent_complete = np.round((index+1)*100/len(lat_long_values), 2)
    if percent_complete - prev_percent_complete > 1:
      print(f'{percent_complete}%', end=' --> ')
      prev_percent_complete = percent_complete

  print('Data fetched!')
  
  # write weather data to pkl file
  out_file = open('/content/drive/MyDrive/Capstone_ML_Guild/Capstone/Time_Series_Analysis_for_Weather/timeseries_weather_data.pkl', 'ab')
  pickle.dump(weather_data, out_file)
  out_file.close()

Weather data found - 459


In [15]:
df_loc = df_crop[['State', 'District', 'Lat', 'Long']].drop_duplicates()

loc_season = []

for i, state, district, lat, lon in df_loc.itertuples():
  for season in season_months.keys():
    loc_season.append([state, district, lat, lon, season])

df_season = pd.DataFrame(loc_season, columns=['State', 'District', 'Lat', 'Long', 'Season'])
df_season

,State,District,Lat,Long,Season
0,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,Whole Year
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,Autumn
2,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,Kharif
3,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,Rabi
4,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,11.66891,92.73273,Summer
...,...,...,...,...,...
3109,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,Autumn
3110,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,Kharif
3111,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,Rabi
3112,WEST BENGAL,WEST MEDINIPUR,22.42410,87.32094,Summer


In [17]:
# Find averages based on season
geolocation_weather_by_year = []

for i, state, district, lat, lon, season in df_season.itertuples():
  months = season_months[season]
  geo_weather = weather_data[lat][lon]
  parameters = list(geo_weather.keys())
  temp_data = {}
  for year in range(1982, 2020):

    # for param in parameters:
    #   required_year = year
    #   if months[-1] - months[0] < 0:
    #     required_year -= 1
    #   last_month = 0
    #   if param not in temp_data:
    #     temp_data[param] = []
    #   for month in months:
    #     if month < last_month:
    #       required_year += 1
    #     required_key = str(str(required_year) + str('%02d' % month))
    #     last_month = month
    #     temp_data[param].append(geo_weather[param][required_key])

    for month in range(1,13):
      required_key = str(str(year) + str('%02d' % month))
      geolocation_weather_by_year.append({
        "State": state,
        "District": district,
        "Lat": lat,
        "Long": lon,
        "Year": year,
        "Month": month,
        "Solar_Radiation": geo_weather['ALLSKY_SFC_SW_DWN'][required_key],
        "Precipitation": geo_weather['PRECTOT'][required_key],
        "Surface_Pressure": geo_weather['PS'][required_key],
        "Humidity": np.round(geo_weather['RH2M'][required_key], 3),
        "Dew_Frost_Point": np.round(geo_weather['T2MDEW'][required_key], 3),
        "Temp_Max": np.round(geo_weather['T2M_MAX'][required_key], 3),
        "Temp_Min": np.round(geo_weather['T2M_MIN'][required_key], 3),
        "Wind_Speed": np.round(geo_weather['WS2M'][required_key], 3)
      })

df_weather = pd.DataFrame(geolocation_weather_by_year)

In [18]:
df_weather = df_weather.sort_values('Year').reset_index(drop=True)
df_weather.shape

(1419984, 14)

In [19]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419984 entries, 0 to 1419983
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   State             1419984 non-null  object 
 1   District          1419984 non-null  object 
 2   Lat               1419984 non-null  float64
 3   Long              1419984 non-null  float64
 4   Year              1419984 non-null  int64  
 5   Month             1419984 non-null  int64  
 6   Solar_Radiation   1419984 non-null  float64
 7   Precipitation     1419984 non-null  float64
 8   Surface_Pressure  1419984 non-null  float64
 9   Humidity          1419984 non-null  float64
 10  Dew_Frost_Point   1419984 non-null  float64
 11  Temp_Max          1419984 non-null  float64
 12  Temp_Min          1419984 non-null  float64
 13  Wind_Speed        1419984 non-null  float64
dtypes: float64(10), int64(2), object(2)
memory usage: 151.7+ MB


In [20]:
df_weather = df_weather.drop_duplicates()
df_weather.to_csv('/content/drive/MyDrive/Capstone_ML_Guild/Capstone/Time_Series_Analysis_for_Weather/Weather_TimeSeries.csv', index=False)